In [3]:
import pandas as pd
import os
import numpy as np

In [4]:
# Construimos la ruta dinámica al archivo
file_path = os.path.join("..", "data", "citas_medicas.csv")

# Leemos el CSV
df = pd.read_csv(file_path)

# Verificamos que cargó mostrando las primeras 5 filas
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [5]:
nuevos_nombres = {
    'AppointmentID': 'appointment_id',
    'Gender': 'gender',
    'ScheduledDay': 'scheduled_day',   # Día que llamó para reservar
    'AppointmentDay': 'appointment_day', # Día de la cita médica real
    'Age': 'age',
    'Neighbourhood': 'neighbourhood',
    'Scholarship': 'scholarship',        # Si tiene seguro social/beca
    'Hipertension': 'hypertension',
    'Diabetes': 'diabetes',
    'Alcoholism': 'alcoholism',
    'Handcap': 'handicap',
    'SMS_received': 'sms_received',
    'No-show': 'no_show'                 # Target: ¿Faltó?
}

# Aplicamos el cambio al DataFrame original (inplace=True)
df.rename(columns=nuevos_nombres, inplace=True)

# Verificamos solo los nombres de columnas
df.columns

Index(['PatientId', 'appointment_id', 'gender', 'scheduled_day',
       'appointment_day', 'age', 'neighbourhood', 'scholarship',
       'hypertension', 'diabetes', 'alcoholism', 'handicap', 'sms_received',
       'no_show'],
      dtype='object')

In [6]:
# Convertir texto a objetos de fecha reales
df['scheduled_day'] = pd.to_datetime(df['scheduled_day']).dt.normalize()
df['appointment_day'] = pd.to_datetime(df['appointment_day']).dt.normalize()

# Verificar los tipos de datos (deberías ver datetime64[ns])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   PatientId        110527 non-null  float64            
 1   appointment_id   110527 non-null  int64              
 2   gender           110527 non-null  object             
 3   scheduled_day    110527 non-null  datetime64[ns, UTC]
 4   appointment_day  110527 non-null  datetime64[ns, UTC]
 5   age              110527 non-null  int64              
 6   neighbourhood    110527 non-null  object             
 7   scholarship      110527 non-null  int64              
 8   hypertension     110527 non-null  int64              
 9   diabetes         110527 non-null  int64              
 10  alcoholism       110527 non-null  int64              
 11  handicap         110527 non-null  int64              
 12  sms_received     110527 non-null  int64              
 13 

In [7]:
# 1. ¿Cuántos días pasaron entre la reserva y la cita?
df['days_wait'] = (df['appointment_day'] - df['scheduled_day']).dt.days

# 2. ¿Qué día de la semana es la cita? (0=Lunes, 6=Domingo)
df['day_of_week'] = df['appointment_day'].dt.dayofweek

# 3. Convertir el objetivo a binario (1 = Faltó / No-Show, 0 = Asistió)
# Lambda es una forma rápida de escribir una función en una línea
df['target'] = df['no_show'].apply(lambda x: 1 if x == 'Yes' else 0)

# Miremos cómo quedaron estas columnas nuevas
df[['scheduled_day', 'appointment_day', 'days_wait', 'day_of_week', 'target']].head()

,scheduled_day,appointment_day,days_wait,day_of_week,target
0,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,4,0
1,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,4,0
2,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,4,0
3,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,4,0
4,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,4,0


In [8]:
print(f"Filas antes de limpiar: {df.shape[0]}")

# Solo aceptamos edad positiva y días de espera positivos (o cero)
df_clean = df[(df['age'] >= 0) & (df['days_wait'] >= 0)].copy()

print(f"Filas después de limpiar: {df_clean.shape[0]}")
# Verás que se eliminan unas cuantas filas erróneas

Filas antes de limpiar: 110527
Filas después de limpiar: 110521


In [9]:
output_path = os.path.join("..", "data", "processed_citas.csv")

# index=False significa "no guardes el número de fila 0,1,2,3 en el archivo"
df_clean.to_csv(output_path, index=False)

print(f"¡Listo! Archivo guardado en: {output_path}")

¡Listo! Archivo guardado en: ..\data\processed_citas.csv
